In [ ]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/drive']

# The ID of a sample presentation.
PRESENTATION_ID = '1l-rOHVLwhzAj8sLD3M0Lj0Vujw4Z4SboW1ImUDcbExI'

creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server()
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service_slides = build('slides', 'v1', credentials=creds)
presentation = service_slides.presentations().get(
    presentationId=PRESENTATION_ID).execute()
slides = presentation.get('slides')

service_sheets = build('sheets', 'v4', credentials=creds)
spreadsheet = {
    'properties': {
        'title': "Aroolla Translate"
    }
}
spreadsheet = service_sheets.spreadsheets().create(body=spreadsheet,
                                    fields='spreadsheetId').execute()
print('Spreadsheet ID: {0}'.format(spreadsheet.get('spreadsheetId')))
    
print('The presentation contains {} slides'.format(len(slides)))


In [ ]:
from google.cloud import translate

translate_client = translate.Client.from_service_account_json('tts.json')

def google_translate(text):
    target = 'en'
    source = 'fr'
    translation = translate_client.translate(text, target_language=target, source_language=source)
    return translation['translatedText']

In [ ]:
ROW_ID = 1
alphabet = ['A', 'B', 'C', 'D', 'E', 'F', 'G']

def write_sheet(values):
    global ROW_ID
    body = {
        'values': [values]
    }
    #print(body)
    service_sheets.spreadsheets().values().update(
        spreadsheetId=spreadsheet.get('spreadsheetId'), range='A{0}:{1}{0}'.format(ROW_ID, alphabet[len(values)-1]),
        valueInputOption='RAW', body=body).execute()
    ROW_ID += 1

In [ ]:
def parse_table(slide_nr, slide_id, element):
    table = element['table']
    for row in table['tableRows']:
        for cell in row['tableCells']:
            try:
                for text_element in cell['text']['textElements']:
                    if 'textRun' in text_element:
                        text = text_element['textRun']['content'].rstrip()
                        if text != "":
                            print(element['objectId'], text)
                            write_sheet([slide_nr, slide_id, element['objectId'], text, google_translate(text)])
            except KeyError:
                pass
        
def parse_shape(slide_nr, slide_id, element):
    try:
        text_elements = element['shape']['text']['textElements']
        for text_element in text_elements:
            if 'textRun' in text_element:
                text = text_element['textRun']['content'].rstrip()
                if text != "":
                    print(element['objectId'], text)
                    write_sheet([slide_nr, slide_id, element['objectId'], text, google_translate(text)])
    except KeyError:
        print(element['shape']['shapeType'])

write_sheet(['slide_nr', 'slide_id', 'object_id', 'original_text', 'translated_text'])

for i, slide in enumerate(slides):
    elements = slide.get('pageElements')
    slide_nr = i + 1
    slide_id = slide['objectId']
    print('- Slide #{} contains {} elements.'.format(
            slide_nr, len(elements)))
    for element in elements:
        if 'shape' in element:
            parse_shape(slide_nr, slide_id, element)
        elif 'table' in element:
            parse_table(slide_nr, slide_id, element)


In [ ]:
service_drive = build('drive', 'v3', credentials=creds)
body = {
    'name': 'aroolla_roasteria_intro_EN_final'
}
drive_response = service_drive.files().copy(
    fileId=PRESENTATION_ID, body=body).execute()
presentation_copy_id = drive_response.get('id')

In [ ]:
def translate_element(values_dict):
    requests = []
    for values in values_dict:
        requests.append({'replaceAllText': 
                         {'containsText': {'text': values[3], 'matchCase': True}, 
                          'replaceText': values[4],
                          'pageObjectIds': [values[1]]}}
                       )
    body = {
        'requests': requests
    }
    print(requests)
    response = service_slides.presentations().batchUpdate(
        presentationId=presentation_copy_id, body=body).execute()

    num_replacements = 0
    for reply in response.get('replies'):
        try:
            num_replacements += reply.get('replaceAllText').get('occurrencesChanged')
        except:
            pass
    print('Replaced %d text instances' % num_replacements)

In [ ]:
ROW_ID = 2
running = True
values_dict = []
slide_id = None

while running: 
    result = service_sheets.spreadsheets().values().get(
        spreadsheetId=spreadsheet.get('spreadsheetId'), range='A{0}:E{0}'.format(ROW_ID)).execute()
    numRows = result.get('values')
    if numRows:
        values = numRows[0]
        if len(values) == 5:
            if not slide_id:
                slide_id = values[1]
                values_dict.append(values)
            elif slide_id != values[1]:
                translate_element(values_dict)
                slide_id = values[1]
                values_dict = [values]
            else:
                values_dict.append(values)
        ROW_ID += 1
    else:
        translate_element(values_dict)
        running = False